<a href="https://colab.research.google.com/github/MaiKhoa0101/MachineLearning/blob/main/NewDetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall mediapipe -y
!pip cache purge
!pip install mediapipe==0.10.18
!pip install mediapipe

Found existing installation: mediapipe 0.10.18
Uninstalling mediapipe-0.10.18:
  Successfully uninstalled mediapipe-0.10.18
Files removed: 6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 51.3 MB/s eta 0:00:00


In [ ]:
pip install flask-cors

In [ ]:
"""
COMPLETE SIGN LANGUAGE DETECTION API WITH AUTO-TRIM
====================================================
Tích hợp đầy đủ: Motion Retargeting + Video Trimming + Flask API
"""

# !pip install -q pyngrok flask flask-cors opencv-python mediapipe numpy

## API
!pip install -q pyngrok flask flask-cors

# api_sign_language.py
import os
import json
import tempfile
from pathlib import Path
import uuid
import threading


import os
import json
import tempfile
import cv2
import mediapipe as mp
import numpy as np
import urllib.request
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from pathlib import Path
import uuid
import threading
from datetime import datetime
from math import sqrt, atan2, degrees, acos
from typing import Dict, Optional, Tuple
from pyngrok import ngrok, conf

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def is_url(path: str) -> bool:
    """Kiểm tra xem đường dẫn có phải là URL không"""
    return path.startswith(('http://', 'https://'))

def download_video(url: str) -> str:
    """Download video từ URL và lưu vào file tạm"""
    print(f"🌐 Downloading video from URL: {url}")
    try:
        temp_dir = tempfile.gettempdir()
        temp_filename = f"video_{os.path.basename(url).split('?')[0]}"
        temp_path = os.path.join(temp_dir, temp_filename)

        if not temp_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            temp_path += '.mp4'

        urllib.request.urlretrieve(url, temp_path)
        print(f"✅ Video downloaded to: {temp_path}")
        return temp_path
    except Exception as e:
        print(f"❌ Error downloading video: {e}")
        raise

# ============================================================================
# RIG CONFIGURATION
# ============================================================================
class RigConfiguration:
    """Cấu hình ánh xạ giữa góc vật lý và trục xương của Model 3D"""

    def __init__(self):
        # UPPER ARM
        self.UPPERARM_L = {
            "y": {"axis": "y", "scale": 1.0, "offset": 0},
            "z": {"axis": "z", "scale": 1.0, "offset": 0},
            "x": {"axis": "x", "scale": -1.0, "offset": 0}
        }
        self.UPPERARM_R = {
            "y": {"axis": "y", "scale": 1.0, "offset": 0},
            "z": {"axis": "z", "scale": 1.0, "offset": 0},
            "x": {"axis": "x", "scale": -1.0, "offset": 0}
        }

        # LOWER ARM
        self.LOWERARM_L = {
            "bend": {"axis": "z", "scale": -1.0, "offset": 0},
            "rotation": {"axis": "x", "scale": 1.0, "offset": 0}
        }
        self.LOWERARM_R = {
            "bend": {"axis": "z", "scale": -1.0, "offset": 0},
            "rotation": {"axis": "x", "scale": -1.0, "offset": 0}
        }

        # WRIST/HAND - ✅ CẬP NHẬT ĐẦY ĐỦ 3-DOF
        self.HAND_L = {
            "pitch": {"axis": "z", "scale": 1.0, "offset": 0},   # Cúi/ngửa cổ tay
            "yaw": {"axis": "y", "scale": 0.5, "offset": 0},     # Quay trái/phải
            "roll": {"axis": "x", "scale": 1.0, "offset": 0}     # Xoay (pronation/supination)
        }
        self.HAND_R = {
            "pitch": {"axis": "z", "scale": 1.0, "offset": 0},
            "yaw": {"axis": "y", "scale": 0.5, "offset": 0},
            "roll": {"axis": "x", "scale": 1.0, "offset": 0}
        }

        # FINGERS
        self.FINGER_CONFIG = {"bend": {"axis": "z", "scale": 1.0, "offset": 0}}

        # HEAD & NECK
        self.HEAD = {
            "pitch": {"axis": "z", "scale": 1.0, "offset": 0},
            "yaw": {"axis": "x", "scale": 1.0, "offset": 0},
            "roll": {"axis": "y", "scale": 1.0, "offset": 0}
        }
        self.NECK = {
            "pitch": {"axis": "z", "scale": 0.5, "offset": 0},
            "yaw": {"axis": "x", "scale": 0.5, "offset": 0}
        }

        # SPINE
        self.SPINE = {
            "pitch": {"axis": "z", "scale": 1.0, "offset": 0},
            "yaw": {"axis": "x", "scale": 1.0, "offset": 0}
        }

    def apply_mapping(self, physical_angles: Dict, config: Dict) -> Dict:
        """Áp dụng configuration để chuyển góc vật lý thành output Model"""
        result = {"x": 0, "y": 0, "z": 0}
        for angle_type, angle_value in physical_angles.items():
            if angle_type in config:
                mapping = config[angle_type]
                axis = mapping["axis"]
                scale = mapping["scale"]
                offset = mapping["offset"]
                result[axis] = round(angle_value * scale + offset, 2)
        return result

# ============================================================================
# PHYSICAL ANGLE CALCULATOR
# ============================================================================

class PhysicalAngleCalculator:
    """Tính góc vật lý thực tế từ MediaPipe landmarks"""

    @staticmethod
    def calculate_elbow_bend(shoulder, elbow, wrist) -> float:
        """Tính góc gập khuỷu tay (Law of Cosines)"""
        a = np.linalg.norm([elbow.x - shoulder.x, elbow.y - shoulder.y, elbow.z - shoulder.z])
        b = np.linalg.norm([wrist.x - elbow.x, wrist.y - elbow.y, wrist.z - elbow.z])
        c = np.linalg.norm([wrist.x - shoulder.x, wrist.y - shoulder.y, wrist.z - shoulder.z])

        cos_angle = (a*a + b*b - c*c) / (2 * a * b + 1e-6)
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        angle = degrees(acos(cos_angle))
        return angle

    @staticmethod
    def calculate_forearm_rotation(shoulder, elbow, wrist) -> float:
        """Tính góc xoay cẳng tay"""
        upper_arm = np.array([
            elbow.x - shoulder.x,
            elbow.y - shoulder.y,
            elbow.z - shoulder.z
        ])
        upper_arm = upper_arm / (np.linalg.norm(upper_arm) + 1e-6)

        forearm = np.array([
            wrist.x - elbow.x,
            wrist.y - elbow.y,
            wrist.z - elbow.z
        ])
        forearm = forearm / (np.linalg.norm(forearm) + 1e-6)

        projection = np.dot(forearm, upper_arm) * upper_arm
        perpendicular = forearm - projection
        perp_length = np.linalg.norm(perpendicular) + 1e-6
        perpendicular = perpendicular / perp_length

        reference = np.array([0, 1, 0])
        ref_proj = reference - np.dot(reference, upper_arm) * upper_arm
        ref_length = np.linalg.norm(ref_proj) + 1e-6
        ref_proj = ref_proj / ref_length

        cos_angle = np.clip(np.dot(perpendicular, ref_proj), -1.0, 1.0)
        rotation_angle = degrees(acos(cos_angle))

        cross = np.cross(ref_proj, perpendicular)
        if np.dot(cross, upper_arm) < 0:
            rotation_angle = -rotation_angle

        return round(rotation_angle, 2)

    @staticmethod
    def calculate_wrist_orientation(shoulder, elbow, wrist, hand_landmarks) -> Dict[str, float]:
        """
        ✅ TÍNH GÓC CỔ TAY - SIMPLE Y-AXIS ANGLE

        Chỉ dùng Y component để tính flexion/extension
        """
        if hand_landmarks is None:
            return {"pitch": 0.0, "yaw": 0.0, "roll": 0.0}

        wrist_lm = hand_landmarks.landmark[0]     # Wrist
        middle_mcp = hand_landmarks.landmark[9]   # Middle base

        # Chỉ quan tâm đến Y difference (vertical)
        dy = middle_mcp.y - wrist_lm.y

        # Tính góc từ Y component
        # Trong MediaPipe hand: Y tăng = xuống dưới
        # dy > 0 = hand pointing down (flexion)
        # dy < 0 = hand pointing up (extension)

        # Scale xuống để có góc hợp lý (dựa vào thực nghiệm)
        pitch = dy * 100  # Adjust multiplier as needed

        # Clamp
        pitch = np.clip(pitch, -45, 90)

        return {
            "pitch": round(pitch, 2),
            "yaw": 0.0,
            "roll": 0.0
        }

    @staticmethod
    def calculate_arm_orientation(shoulder, elbow, wrist=None) -> Dict[str, float]:
        """Tính hướng của cánh tay theo Euler XYZ"""
        dx = elbow.x - shoulder.x
        dy = elbow.y - shoulder.y
        dz = elbow.z - shoulder.z

        length = sqrt(dx*dx + dy*dy + dz*dz)
        dx, dy, dz = dx/length, dy/length, dz/length

        y_angle = degrees(dy)
        horizontal_dist = sqrt(dx*dx + dz*dz)
        z_angle = degrees(np.arcsin(dz / horizontal_dist)) + 15
        x_angle = degrees(dx)*-2

        if wrist is not None:
          wx = wrist.x - elbow.x
          wy = wrist.y - elbow.y
          wz = wrist.z - elbow.z

          arm_axis = np.array([dx, dy, dz])
          wrist_vec = np.array([wx, wy, wz])

          projection = np.dot(wrist_vec, arm_axis) * arm_axis
          perpendicular = wrist_vec - projection

          perp_length = np.linalg.norm(perpendicular) + 1e-6
          perpendicular = perpendicular / perp_length

          ref_vec = np.array([0, -1, 0])
          ref_proj = ref_vec - np.dot(ref_vec, arm_axis) * arm_axis
          ref_length = np.linalg.norm(ref_proj) + 1e-6
          ref_proj = ref_proj / ref_length

          cos_angle = np.clip(np.dot(perpendicular, ref_proj), -1.0, 1.0)
          x_angle = degrees(acos(cos_angle))

          cross = np.cross(ref_proj, perpendicular)
          if np.dot(cross, arm_axis) < 0:
              x_angle = -x_angle

        return {
            "y": round(y_angle, 2),
            "z": round(z_angle, 2),
            "x": round(x_angle, 2)
        }

    @staticmethod
    def calculate_finger_bend(landmarks, finger_ids) -> float:
        """Tính góc gập trung bình của ngón tay"""
        total_angle = 0
        count = 0

        for i in range(1, len(finger_ids)-1):
            p1 = landmarks[finger_ids[i-1]]
            p2 = landmarks[finger_ids[i]]
            p3 = landmarks[finger_ids[i+1]]

            a = np.linalg.norm([p2.x - p1.x, p2.y - p1.y, p2.z - p1.z])
            b = np.linalg.norm([p3.x - p2.x, p3.y - p2.y, p3.z - p2.z])
            c = np.linalg.norm([p3.x - p1.x, p3.y - p1.y, p3.z - p1.z])

            cos_angle = (a*a + b*b - c*c) / (2 * a * b + 1e-6)
            cos_angle = np.clip(cos_angle, -1.0, 1.0)

            angle = degrees(acos(cos_angle))
            total_angle += (180 - angle)
            count += 1

        avg_angle = total_angle / count if count > 0 else 0
        return round(avg_angle, 2)

# ============================================================================
# GESTURE DETECTOR
# ============================================================================

class GestureDetector:
    """Detector sử dụng Physical Calculation + Rig Configuration"""

    def __init__(self):
        self.calc = PhysicalAngleCalculator()
        self.rig = RigConfiguration()

    def detect_arm(self, pose_lm, hand_landmarks, side='left') -> Dict:
        """Phát hiện và chuyển đổi góc cánh tay - ✅ ĐÃ CÓ WRIST ORIENTATION"""
        if side == 'left':
            shoulder, elbow, wrist = pose_lm[11], pose_lm[13], pose_lm[15]
            suffix = 'l'
            upperarm_config = self.rig.UPPERARM_L
            lowerarm_config = self.rig.LOWERARM_L
            hand_config = self.rig.HAND_L
        else:
            shoulder, elbow, wrist = pose_lm[12], pose_lm[14], pose_lm[16]
            suffix = 'r'
            upperarm_config = self.rig.UPPERARM_R
            lowerarm_config = self.rig.LOWERARM_R
            hand_config = self.rig.HAND_R

        # Calculate angles
        upperarm_angles = self.calc.calculate_arm_orientation(shoulder, elbow)

        if hand_landmarks is None:
            elbow_bend = 0
            forearm_rotation = 0
            wrist_angles = {"pitch": 0, "yaw": 0, "roll": 0}
        else:
            elbow_bend = self.calc.calculate_elbow_bend(shoulder, elbow, wrist)
            forearm_rotation = self.calc.calculate_forearm_rotation(shoulder, elbow, wrist)
            # ✅ FIXED: Truyền đầy đủ shoulder, elbow, wrist
            wrist_angles = self.calc.calculate_wrist_orientation(
                shoulder, elbow, wrist, hand_landmarks
            )
        # Apply rig configuration
        upperarm_result = self.rig.apply_mapping(upperarm_angles, upperarm_config)

        lowerarm_result = self.rig.apply_mapping({
            "bend": elbow_bend,
            "rotation": forearm_rotation
        }, lowerarm_config)

        # ✅ MAP WRIST ORIENTATION
        hand_result = self.rig.apply_mapping(wrist_angles, hand_config)

        output = {
            f"shoulder_{suffix}": f"shoulder_{suffix}(x=0, y=0, z=0)",
            f"upperarm_{suffix}": f"upperarm_{suffix}(x={upperarm_result['x']*2 if suffix=='r' else upperarm_result['x']*-2}, y={upperarm_result['y']}, z={upperarm_result['z']})",
            f"lowerarm_{suffix}": f"lowerarm_{suffix}(x={lowerarm_result['x']}, y={lowerarm_result['y']}, z={lowerarm_result['z']})",
            # ✅ WRIST ORIENTATION ĐẦY ĐỦ
            f"hand_{suffix}": f"hand_{suffix}(x={hand_result['x']}, y={hand_result['y']}, z={hand_result['z']})"
        }

        # Fingers
        if hand_landmarks is None:
            for finger in ['thumb', 'index', 'middle', 'ring', 'pinky']:
                output[f"{finger}_{suffix}"] = self._default_finger(finger, suffix)
        else:
            hand_lm = hand_landmarks.landmark

            finger_ids = {
                "thumb": [1, 2, 3, 4],
                "index": [5, 6, 7, 8],
                "middle": [9, 10, 11, 12],
                "ring": [13, 14, 15, 16],
                "pinky": [17, 18, 19, 20]
            }

            for fname, ids in finger_ids.items():
                bend = self.calc.calculate_finger_bend(hand_lm, ids)
                finger_result = self.rig.apply_mapping({"bend": bend}, self.rig.FINGER_CONFIG)
                output[f"{fname}_{suffix}"] = self._format_finger(fname, suffix, finger_result['z'])

        return output

    def detect_head_neck(self, pose_lm) -> Dict:
        """Phát hiện head & neck"""
        return {
            "head": "head(x=0, y=0, z=0)",
            "neck": "neck(x=0, y=0, z=0)"
        }

    def detect_spine(self, pose_lm) -> Dict:
        """Phát hiện spine"""
        return {
            "spine_01": "spine_01(x=0, y=0, z=0)",
            "spine_02": "spine_02(x=0, y=0, z=0)",
            "spine_03": "spine_03(x=0, y=0, z=0)"
        }

    def _default_finger(self, finger, suffix):
        if finger == 'thumb':
            return f"thumb_02_{suffix}(x=0, y=0, z=0); thumb_03_{suffix}(x=0, y=0, z=0)"
        else:
            return f"{finger}_01_{suffix}(x=0, y=0, z=0); {finger}_02_{suffix}(x=0, y=0, z=0); {finger}_03_{suffix}(x=0, y=0, z=0)"

    def _format_finger(self, finger, suffix, bend_angle):
        if finger == 'thumb':
            return f"thumb_02_{suffix}(x=0, y=0, z={bend_angle}); thumb_03_{suffix}(x=0, y=0, z={bend_angle})"
        else:
            return f"{finger}_01_{suffix}(x=0, y=0, z={bend_angle}); {finger}_02_{suffix}(x=0, y=0, z={bend_angle}); {finger}_03_{suffix}(x=0, y=0, z={bend_angle})"


# ============================================================================
# MAIN DETECTOR
# ============================================================================

class SignLanguageDetector:
    """Main detector với Motion Retargeting"""

    def __init__(self):
        self.mp_holistic = mp.solutions.holistic
        self.holistic = self.mp_holistic.Holistic(
            min_detection_confidence=0.3,
            min_tracking_confidence=0.3,
            model_complexity=1
        )
        self.detector = GestureDetector()
        print("✅ SignLanguageDetector V9 (WITH WRIST ORIENTATION) initialized")

    def process_frame(self, frame) -> Tuple[Optional[Dict], Optional[np.ndarray]]:
        """Process frame and return gesture data + skeleton image"""
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.holistic.process(img_rgb)

        if not results.pose_landmarks:
            return None, None

        # Draw skeleton
        skeleton_img = self.draw_skeleton(frame, results)

        pose_lm = results.pose_landmarks.landmark

        # Detect all body parts
        output = {}
        output.update(self.detector.detect_spine(pose_lm))
        output.update(self.detector.detect_head_neck(pose_lm))

        # Facial (default values)
        output.update({
            "jaw": "jaw(z=0)",
            "eyelid_l": "eyelid_l(z=0)",
            "eyelid_r": "eyelid_r(z=0)",
            "eyes": "eye_l(y=0); eye_r(y=0)",
            "eyebrows": "eyebrow_l(z=0); eyebrow_r(z=0)",
            "mouth_l": "mouth_l(x=0, y=0, z=0)",
            "mouth_r": "mouth_r(x=0, y=0, z=0)"
        })

        # Arms (with wrist orientation)
        output.update(self.detector.detect_arm(pose_lm, results.left_hand_landmarks, 'left'))
        output.update(self.detector.detect_arm(pose_lm, results.right_hand_landmarks, 'right'))

        return output, skeleton_img

    def draw_skeleton(self, frame, results):
        """Draw skeleton with wrist orientation indicators"""
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing_styles = mp.solutions.drawing_styles
        mp_holistic = mp.solutions.holistic

        annotated = frame.copy()
        h, w, _ = annotated.shape

        # Draw pose landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                annotated, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )

        pose_lm = results.pose_landmarks.landmark if results.pose_landmarks else None

        # Draw hand landmarks with wrist orientation
        if results.right_hand_landmarks and pose_lm:
            mp_drawing.draw_landmarks(
                annotated, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

            # ✅ VẼ WRIST ORIENTATION CHO TAY PHẢI
            shoulder_r = pose_lm[12]  # Right shoulder
            elbow_r = pose_lm[14]     # Right elbow
            wrist_r = pose_lm[16]     # Right wrist

            wrist_angles = self.detector.calc.calculate_wrist_orientation(
                shoulder_r, elbow_r, wrist_r, results.right_hand_landmarks
            )
            self._draw_wrist_orientation(annotated, wrist_r, w, h, wrist_angles, "R")

        if results.left_hand_landmarks and pose_lm:
            mp_drawing.draw_landmarks(
                annotated, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

            # ✅ VẼ WRIST ORIENTATION CHO TAY TRÁI
            shoulder_l = pose_lm[11]  # Left shoulder
            elbow_l = pose_lm[13]     # Left elbow
            wrist_l = pose_lm[15]     # Left wrist

            wrist_angles = self.detector.calc.calculate_wrist_orientation(
                shoulder_l, elbow_l, wrist_l, results.left_hand_landmarks
            )
            self._draw_wrist_orientation(annotated, wrist_l, w, h, wrist_angles, "L")

        return annotated

    def _draw_wrist_orientation(self, img, wrist_pose, img_w, img_h, angles, side):
        """
        ✅ VẼ CHỈ BÁO WRIST ORIENTATION - FIXED

        Args:
            img: Image to draw on
            wrist_pose: Wrist landmark from pose
            img_w: Image width
            img_h: Image height
            angles: Dict với pitch, yaw, roll
            side: "L" hoặc "R"
        """
        cx = int(wrist_pose.x * img_w)
        cy = int(wrist_pose.y * img_h)

        if cx < 0 or cx >= img_w or cy < 0 or cy >= img_h:
            return

        # Vẽ box hiển thị angles
        box_width = 120
        box_height = 60
        box_x = cx - box_width // 2
        box_y = cy - 80

        # Đảm bảo box không vượt khỏi frame
        box_x = max(0, min(box_x, img_w - box_width))
        box_y = max(0, min(box_y, img_h - box_height))

        # Background
        cv2.rectangle(img,
                    (box_x, box_y),
                    (box_x + box_width, box_y + box_height),
                    (0, 0, 0), -1)
        cv2.rectangle(img,
                    (box_x, box_y),
                    (box_x + box_width, box_y + box_height),
                    (0, 255, 255), 2)

        # Text
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.4
        thickness = 1

        # Title
        cv2.putText(img, f"{side}_Wrist",
                  (box_x + 5, box_y + 15),
                  font, font_scale, (0, 255, 255), thickness)

        # Angles
        cv2.putText(img, f"P:{angles['pitch']:6.1f}",
                  (box_x + 5, box_y + 30),
                  font, font_scale, (255, 255, 255), thickness)
        cv2.putText(img, f"Y:{angles['yaw']:6.1f}",
                  (box_x + 5, box_y + 45),
                  font, font_scale, (255, 255, 255), thickness)
        cv2.putText(img, f"R:{angles['roll']:6.1f}",
                  (box_x + 70, box_y + 30),
                  font, font_scale, (255, 255, 255), thickness)

    def close(self):
        self.holistic.close()

# ============================================================================
# OPTIMIZED VIDEO PROCESSOR WITH AUTO-TRIM
# ============================================================================

class OptimizedVideoProcessor:
    """Xử lý video tối ưu với auto-trim real-time"""

    def __init__(self,
                 movement_threshold: float = 0.02,
                 arm_down_threshold: float = 0.6,
                 min_frames_active: int = 5,
                 buffer_frames: int = 3):
        self.movement_threshold = movement_threshold
        self.arm_down_threshold = arm_down_threshold
        self.min_frames_active = min_frames_active
        self.buffer_frames = buffer_frames

        # MediaPipe
        self.mp_holistic = mp.solutions.holistic
        self.holistic = self.mp_holistic.Holistic(
            min_detection_confidence=0.3,
            min_tracking_confidence=0.3,
            model_complexity=1
        )

        # Gesture Detector
        self.detector = GestureDetector()

        # State tracking
        self.prev_landmarks = None
        self.consecutive_active_frames = 0
        self.started = False
        self.start_frame_id = None

    def calculate_movement(self, current_landmarks, prev_landmarks) -> float:
        """Tính toán tổng chuyển động của cả 2 tay"""
        if prev_landmarks is None:
            return 0.0

        indices = [11, 13, 15, 12, 14, 16]
        total_movement = 0.0

        for idx in indices:
            curr = current_landmarks[idx]
            prev = prev_landmarks[idx]

            dx = curr.x - prev.x
            dy = curr.y - prev.y
            dz = curr.z - prev.z

            distance = np.sqrt(dx*dx + dy*dy + dz*dz)
            total_movement += distance

        return total_movement / len(indices)

    def get_arms_position(self, landmarks) -> float:
        """Lấy vị trí Y trung bình của 2 cổ tay"""
        left_wrist = landmarks[15]
        right_wrist = landmarks[16]
        return (left_wrist.y + right_wrist.y) / 2

    def should_start_processing(self, landmarks) -> bool:
        """Kiểm tra xem có nên bắt đầu xử lý không"""
        if self.started:
            return True

        movement = self.calculate_movement(landmarks, self.prev_landmarks)

        if movement > self.movement_threshold:
            self.consecutive_active_frames += 1

            if self.consecutive_active_frames >= self.min_frames_active:
                self.started = True
                print(f"🎬 START DETECTED! Beginning analysis...")
                return True
        else:
            self.consecutive_active_frames = 0

        return False

    def should_stop_processing(self, landmarks) -> bool:
        """Kiểm tra xem có nên dừng xử lý không"""
        if not self.started:
            return False

        arms_y = self.get_arms_position(landmarks)
        movement = self.calculate_movement(landmarks, self.prev_landmarks)

        arms_down = arms_y > self.arm_down_threshold
        movement_low = movement < self.movement_threshold * 0.5

        if arms_down and movement_low:
            print(f"🛑 END DETECTED! Stopping analysis...")
            return True

        return False

    def process_video_optimized(self,
                                video_source,
                                output_json="gestures_v6.json",
                                output_dir="frames_v6",
                                frames_per_minute=1):
        """Xử lý video với auto-trim tối ưu"""
        temp_file = None

        try:
            # Download nếu là URL
            if is_url(video_source):
                video_path = download_video(video_source)
                temp_file = video_path
            else:
                video_path = video_source
                if not os.path.exists(video_path):
                    raise ValueError(f"Video file not found: {video_path}")

            # Tạo output folders
            Path(output_dir).mkdir(exist_ok=True)
            frames_folder = Path(output_dir) / "original_frames"
            skeleton_folder = Path(output_dir) / "skeleton_frames"
            frames_folder.mkdir(exist_ok=True)
            skeleton_folder.mkdir(exist_ok=True)

            # Open video
            cap = cv2.VideoCapture(video_path)

            if not cap.isOpened():
                raise ValueError(f"Cannot open video: {video_path}")

            fps = cap.get(cv2.CAP_PROP_FPS)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            duration_sec = total_frames / fps if fps > 0 else 0
            frame_interval = int(fps * 0.1 / frames_per_minute) if fps > 0 else 1

            results = []
            frame_id = 0
            detected_count = 0
            skipped_frames = 0

            print(f"\n{'='*70}")
            print(f"🎬 VIDEO: {video_source}")
            print(f"📊 FPS: {fps:.2f} | Duration: {duration_sec:.1f}s")
            print(f"⏱️  Sampling: 1 frame every {frame_interval} frames")
            print(f"🔍 Auto-trim: ENABLED")
            print(f"{'='*70}\n")

            print("🔎 Scanning for start point...")

            # Buffer để lưu frames trước start point
            pre_start_buffer = []

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # PHASE 1: TÌM ĐIỂM BẮT ĐẦU
                if not self.started:
                    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    results_mp = self.holistic.process(img_rgb)

                    if results_mp.pose_landmarks:
                        landmarks = results_mp.pose_landmarks.landmark

                        # Lưu vào buffer
                        if len(pre_start_buffer) < self.buffer_frames:
                            pre_start_buffer.append((frame_id, frame, results_mp))
                        else:
                            pre_start_buffer.pop(0)
                            pre_start_buffer.append((frame_id, frame, results_mp))

                        # Kiểm tra xem có nên bắt đầu không
                        if self.should_start_processing(landmarks):
                            self.start_frame_id = frame_id

                            # Xử lý buffer frames
                            print(f"📦 Processing {len(pre_start_buffer)} buffer frames...")
                            for buf_frame_id, buf_frame, buf_results in pre_start_buffer:
                                gesture_data = self._process_detection(buf_results, buf_frame_id, fps)

                                if gesture_data:
                                    self._save_frame_data(
                                        buf_frame_id, buf_frame,
                                        gesture_data, fps,
                                        frames_folder, skeleton_folder, results
                                    )
                                    detected_count += 1

                        self.prev_landmarks = landmarks

                    skipped_frames += 1

                    if frame_id % 300 == 0:
                        print(f"  ⏳ Scanning... Frame {frame_id}/{total_frames}")

                    frame_id += 1
                    continue

                # PHASE 2: XỬ LÝ SAU KHI ĐÃ BẮT ĐẦU
                if frame_id % frame_interval == 0:
                    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    results_mp = self.holistic.process(img_rgb)

                    if results_mp.pose_landmarks:
                        landmarks = results_mp.pose_landmarks.landmark

                        # Kiểm tra xem có nên dừng không
                        if self.should_stop_processing(landmarks):
                            print(f"✅ Processing complete at frame {frame_id}")
                            break

                        # Xử lý frame
                        gesture_data = self._process_detection(results_mp, frame_id, fps)

                        if gesture_data:
                            self._save_frame_data(
                                frame_id, frame,
                                gesture_data, fps,
                                frames_folder, skeleton_folder, results
                            )
                            detected_count += 1

                            print(f"  ✅ Frame {frame_id:6d} at {frame_id/fps:7.2f}s - Detected #{detected_count}")

                        self.prev_landmarks = landmarks

                frame_id += 1

                if frame_id % 300 == 0:
                    progress = (frame_id / total_frames * 100) if total_frames > 0 else 0
                    print(f"  ⏳ Progress: {frame_id}/{total_frames} ({progress:.1f}%)")

            cap.release()
            self.holistic.close()

            # Save JSON
            with open(output_json, "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=2)

            print(f"\n{'='*70}")
            print(f"✅ COMPLETED!")
            print(f"🎬 Start frame: {self.start_frame_id}")
            print(f"⏩ Skipped frames: {skipped_frames}")
            print(f"📦 Detected: {detected_count} frames")
            print(f"💾 JSON: {output_json}")
            print(f"{'='*70}\n")

            return results

        finally:
            if temp_file and os.path.exists(temp_file):
                try:
                    os.remove(temp_file)
                    print(f"🗑️  Cleaned up temporary file: {temp_file}")
                except:
                    pass

    def _process_detection(self, results_mp, frame_id, fps):
        """Xử lý detection cho 1 frame"""
        pose_lm = results_mp.pose_landmarks.landmark

        output = {}
        output.update(self.detector.detect_spine(pose_lm))
        output.update(self.detector.detect_head_neck(pose_lm))

        # Facial (default values)
        output.update({
            "jaw": "jaw(z=0)",
            "eyelid_l": "eyelid_l(z=0)",
            "eyelid_r": "eyelid_r(z=0)",
            "eyes": "eye_l(y=0); eye_r(y=0)",
            "eyebrows": "eyebrow_l(z=0); eyebrow_r(z=0)",
            "mouth_l": "mouth_l(x=0, y=0, z=0)",
            "mouth_r": "mouth_r(x=0, y=0, z=0)"
        })

        # Arms
        output.update(self.detector.detect_arm(pose_lm, results_mp.left_hand_landmarks, 'left'))
        output.update(self.detector.detect_arm(pose_lm, results_mp.right_hand_landmarks, 'right'))

        return output

    def _save_frame_data(self, frame_id, frame, gesture_data, fps,
                        frames_folder, skeleton_folder, results):
        """Lưu frame data"""
        timestamp = frame_id / fps if fps > 0 else frame_id
        frame_filename = f"frame_{frame_id:06d}_t{timestamp:.2f}s.jpg"
        skeleton_filename = f"skeleton_{frame_id:06d}_t{timestamp:.2f}s.jpg"

        cv2.imwrite(str(frames_folder / frame_filename), frame)

        results.append({
            "frame": frame_id,
            "timestamp": round(timestamp, 3),
            "original_image": str(frames_folder / frame_filename),
            "skeleton_image": str(skeleton_folder / skeleton_filename),
            "gestures": gesture_data
        })

# ============================================================================
# FLASK API
# ============================================================================

# Setup ngrok
conf.get_default().auth_token = "3721EzVULUpY9QkshdRLFveW6LV_2hLJT8sRukP7E4hw6zz6Y"

app = Flask(__name__)
CORS(app)

RESULTS_DIR = "api_results"
os.makedirs(RESULTS_DIR, exist_ok=True)

jobs = {}

class DetectionJob:
    """Lớp quản lý job xử lý video"""

    def __init__(self, job_id, video_url):
        self.job_id = job_id
        self.video_url = video_url
        self.status = "pending"
        self.progress = 0
        self.message = ""
        self.result_path = None
        self.frames_per_minute = 1
        self.created_at = datetime.now()
        self.started_at = None
        self.completed_at = None

    def start_processing(self):
        """Bắt đầu xử lý video với auto-trim"""
        self.status = "processing"
        self.started_at = datetime.now()

        try:
            # Tạo thư mục kết quả
            job_dir = os.path.join(RESULTS_DIR, self.job_id)
            os.makedirs(job_dir, exist_ok=True)

            output_json = os.path.join(job_dir, "api-gestures.json")
            output_frames = os.path.join(job_dir, "frames")

            # Xử lý video
            processor = OptimizedVideoProcessor(
                movement_threshold=0.02,
                arm_down_threshold=0.6,
                min_frames_active=5,
                buffer_frames=3
            )

            processor.process_video_optimized(
                video_source=self.video_url,
                output_json=output_json,
                output_dir=output_frames,
                frames_per_minute=self.frames_per_minute
            )

            # Cập nhật trạng thái
            self.status = "completed"
            self.result_path = output_json
            self.completed_at = datetime.now()
            self.message = f"Processing completed"

        except Exception as e:
            self.status = "failed"
            self.completed_at = datetime.now()
            self.message = f"Error: {str(e)}"
            print(f"Job {self.job_id} failed: {e}")

# ============================================================================
# API ENDPOINTS
# ============================================================================

@app.route('/')
def index():
    """Trang chủ API"""
    return jsonify({
        "name": "Sign Language Detection API V6",
        "version": "1.0.0",
        "description": "API for sign language gesture detection with auto-trim",
        "endpoints": {
            "POST /api/detect": "Submit video URL for processing",
            "GET /api/job/<job_id>": "Check job status",
            "GET /api/job/<job_id>/download": "Download results JSON",
            "GET /api/jobs": "List all jobs",
            "GET /api/health": "Health check"
        }
    })

@app.route('/api/detect', methods=['POST'])
def detect_gestures():
    """
    API endpoint để xử lý video từ URL

    Request body:
    {
        "video_url": "https://example.com/video.mp4",
        "frames_per_minute": 1 (optional, default: 1)
    }
    """
    try:
        data = request.get_json()

        if not data or 'video_url' not in data:
            return jsonify({"error": "Missing video_url in request body"}), 400

        video_url = data['video_url'].strip()

        if not (video_url.startswith('http://') or video_url.startswith('https://')):
            return jsonify({"error": "Invalid URL. Must start with http:// or https://"}), 400

        # Tạo job ID
        job_id = str(uuid.uuid4())

        # Tạo job mới
        job = DetectionJob(job_id, video_url)

        # Cập nhật frames_per_minute nếu có
        if 'frames_per_minute' in data:
            try:
                fpm = float(data['frames_per_minute'])
                if 0.1 <= fpm <= 60:
                    job.frames_per_minute = fpm
            except:
                pass

        # Lưu job
        jobs[job_id] = job

        # Chạy xử lý trong thread riêng
        thread = threading.Thread(target=job.start_processing)
        thread.daemon = True
        thread.start()

        # Trả về thông tin job
        return jsonify({
            "job_id": job_id,
            "status": job.status,
            "message": "Job created and processing started",
            "video_url": video_url,
            "created_at": job.created_at.isoformat(),
            "check_status_url": f"/api/job/{job_id}",
            "download_url": f"/api/job/{job_id}/download"
        }), 202

    except Exception as e:
        return jsonify({"error": f"Server error: {str(e)}"}), 500

@app.route('/api/job/<job_id>', methods=['GET'])
def get_job_status(job_id):
    """
    Kiểm tra trạng thái của job

    Response:
    {
        "job_id": "uuid",
        "status": "pending|processing|completed|failed",
        "progress": 0-100,
        "message": "Status message",
        "video_url": "original_url",
        "created_at": "timestamp",
        "started_at": "timestamp",
        "completed_at": "timestamp",
        "result_available": true/false
    }
    """
    if job_id not in jobs:
        return jsonify({"error": f"Job {job_id} not found"}), 404

    job = jobs[job_id]

    # Tính progress
    if job.status == "processing":
        job.progress = 50  # Có thể cải tiến thêm
    elif job.status == "completed":
        job.progress = 100

    return jsonify({
        "job_id": job.job_id,
        "status": job.status,
        "progress": job.progress,
        "message": job.message,
        "video_url": job.video_url,
        "frames_per_minute": job.frames_per_minute,
        "created_at": job.created_at.isoformat() if job.created_at else None,
        "started_at": job.started_at.isoformat() if job.started_at else None,
        "completed_at": job.completed_at.isoformat() if job.completed_at else None,
        "result_available": job.status == "completed" and job.result_path is not None
    })

@app.route('/api/job/<job_id>/download', methods=['GET'])
def download_results(job_id):
    """Tải về file JSON kết quả"""
    if job_id not in jobs:
        return jsonify({"error": f"Job {job_id} not found"}), 404

    job = jobs[job_id]

    if job.status != "completed":
        return jsonify({"error": f"Job is not completed yet. Current status: {job.status}"}), 400

    if not job.result_path or not os.path.exists(job.result_path):
        return jsonify({"error": "Result file not found"}), 404

    return send_file(
        job.result_path,
        as_attachment=True,
        download_name=f"gestures_{job_id}.json",
        mimetype='application/json'
    )

@app.route('/api/job/<job_id>/preview', methods=['GET'])
def preview_results(job_id):
    """
    Xem trước kết quả (lấy một vài frame đầu tiên)

    Query parameters:
    - limit: số lượng frame muốn xem (default: 5)
    """
    if job_id not in jobs:
        return jsonify({"error": f"Job {job_id} not found"}), 404

    job = jobs[job_id]

    if job.status != "completed":
        return jsonify({"error": f"Job is not completed yet. Current status: {job.status}"}), 400

    if not job.result_path or not os.path.exists(job.result_path):
        return jsonify({"error": "Result file not found"}), 404

    try:
        with open(job.result_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        limit = request.args.get('limit', default=5, type=int)
        preview_data = data[:limit] if isinstance(data, list) else data

        return jsonify({
            "job_id": job_id,
            "total_frames": len(data) if isinstance(data, list) else 1,
            "preview_limit": limit,
            "preview_data": preview_data
        })
    except Exception as e:
        return jsonify({"error": f"Error reading result file: {str(e)}"}), 500

@app.route('/api/jobs', methods=['GET'])
def list_jobs():
    """
    Liệt kê tất cả jobs

    Query parameters:
    - status: lọc theo status (pending, processing, completed, failed)
    - limit: số lượng job tối đa (default: 50)
    """
    status_filter = request.args.get('status')
    limit = request.args.get('limit', default=50, type=int)

    filtered_jobs = []
    for job_id, job in jobs.items():
        if status_filter and job.status != status_filter:
            continue

        filtered_jobs.append({
            "job_id": job_id,
            "status": job.status,
            "video_url": job.video_url,
            "created_at": job.created_at.isoformat() if job.created_at else None,
            "completed_at": job.completed_at.isoformat() if job.completed_at else None,
            "result_available": job.status == "completed" and job.result_path is not None
        })

    # Sắp xếp theo thời gian tạo (mới nhất trước)
    filtered_jobs.sort(key=lambda x: x['created_at'] if x['created_at'] else '', reverse=True)
    filtered_jobs = filtered_jobs[:limit]

    return jsonify({
        "total_jobs": len(jobs),
        "filtered_count": len(filtered_jobs),
        "jobs": filtered_jobs
    })

@app.route('/api/job/<job_id>', methods=['DELETE'])
def delete_job(job_id):
    """Xóa job và các file liên quan"""
    if job_id not in jobs:
        return jsonify({"error": f"Job {job_id} not found"}), 404

    job = jobs[job_id]

    try:
        # Xóa thư mục kết quả nếu có
        job_dir = os.path.join(RESULTS_DIR, job_id)
        if os.path.exists(job_dir):
            import shutil
            shutil.rmtree(job_dir)

        # Xóa job khỏi dictionary
        del jobs[job_id]

        return jsonify({"message": f"Job {job_id} deleted successfully"})
    except Exception as e:
        return jsonify({"error": f"Error deleting job: {str(e)}"}), 500

@app.route('/api/health', methods=['GET'])
def health_check():
    """Endpoint kiểm tra tình trạng API"""
    return jsonify({
        "status": "healthy",
        "timestamp": datetime.now().isoformat(),
        "active_jobs": len([j for j in jobs.values() if j.status == "processing"]),
        "total_jobs": len(jobs)
    })

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def cleanup_old_jobs():
    """Xóa các jobs cũ hơn 24 giờ"""
    try:
        cutoff_time = datetime.now().timestamp() - (24 * 3600)
        jobs_to_delete = []

        for job_id, job in jobs.items():
            if job.completed_at and job.completed_at.timestamp() < cutoff_time:
                jobs_to_delete.append(job_id)

        for job_id in jobs_to_delete:
            try:
                job_dir = os.path.join(RESULTS_DIR, job_id)
                if os.path.exists(job_dir):
                    import shutil
                    shutil.rmtree(job_dir)
                del jobs[job_id]
            except:
                pass

        print(f"Cleaned up {len(jobs_to_delete)} old jobs")
    except Exception as e:
        print(f"Error in cleanup: {e}")

@app.before_request
def log_request():
    """Middleware để log request"""
    if request.path not in ['/health', '/']:
        print(f"[{datetime.now().isoformat()}] {request.method} {request.path}")

# ============================================================================
# MAIN
# ============================================================================

if __name__ == '__main__':
    print("\n" + "="*70)
    print("SIGN LANGUAGE DETECTION API V6")
    print("="*70)

    # Cleanup jobs cũ
    cleanup_old_jobs()

    # Tắt các tunnel ngrok cũ
    try:
        ngrok.kill()
    except:
        pass

    # Tạo tunnel ngrok
    try:
        public_url = ngrok.connect(5000)
        print(f"🌍 PUBLIC URL: {public_url}")
    except Exception as e:
        print(f"⚠️  Ngrok error: {e}")
        print("Running in local mode only")

    print(f"🏠 Local URL: http://localhost:5000")
    print(f"📁 Results directory: {RESULTS_DIR}")
    print("\n📌 Endpoints:")
    print("  POST /api/detect              - Submit video URL")
    print("  GET  /api/job/<id>            - Check job status")
    print("  GET  /api/job/<id>/download   - Download results")
    print("  GET  /api/job/<id>/preview    - Preview results")
    print("  GET  /api/jobs                - List all jobs")
    print("  DELETE /api/job/<id>          - Delete job")
    print("  GET  /api/health              - Health check")
    print("="*70 + "\n")

    # Chạy server
    app.run(host='0.0.0.0', port=5000, debug=False)


SIGN LANGUAGE DETECTION API V6
Cleaned up 0 old jobs
🌍 PUBLIC URL: NgrokTunnel: "https://lorriane-noncongregative-benson.ngrok-free.dev" -> "http://localhost:5000"
🏠 Local URL: http://localhost:5000
📁 Results directory: api_results

📌 Endpoints:
  POST /api/detect              - Submit video URL
  GET  /api/job/<id>            - Check job status
  GET  /api/job/<id>/download   - Download results
  GET  /api/job/<id>/preview    - Preview results
  GET  /api/jobs                - List all jobs
  DELETE /api/job/<id>          - Delete job
  GET  /api/health              - Health check

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
